In [1]:
import dataset
import numpy as np
import tensorflow as tf
from tqdm import tqdm

In [2]:
class Model:
    
    input_features = 100
    batch_size = 128
    epochs = 1
    gan = tf.keras.Sequential()
    image_width = 64
    dataset = []
    learning_rate = 0.001
    beta_1 = 0.05
    
    def random_generator_input(self):
        return tf.random.normal((self.batch_size, self.input_features))
    
    def GAN(self):
        
        #generator
        generator = tf.keras.Sequential()
        generator.add(tf.keras.layers.Dense(self.input_features*4*4, input_shape=[self.input_features]))
        generator.add(tf.keras.layers.Reshape([4, 4, self.input_features]))
        generator.add(tf.keras.layers.Conv2DTranspose(int(self.input_features / 2), kernel_size = 5, strides = 2, padding = 'same', activation = 'selu'))
        generator.add(tf.keras.layers.Conv2DTranspose(int(self.input_features / 4), kernel_size = 10, strides = 4, padding = 'same', activation = 'selu'))
        generator.add(tf.keras.layers.Conv2DTranspose(3, kernel_size = 10, strides = 2, padding = 'same', activation = 'sigmoid'))
        generator.summary()
        
        #discriminator
        discriminator = tf.keras.Sequential()
        discriminator.add(tf.keras.layers.Conv2D(200, kernel_size = 4, strides = 1, padding = 'same', input_shape= [self.image_width, self.image_width, 3]))
        discriminator.add(tf.keras.layers.Conv2D(100, kernel_size=5, strides = 2, padding = 'same'))
        discriminator.add(tf.keras.layers.Conv2D(9, kernel_size = 5, strides = 2, padding = 'same'))
        discriminator.add(tf.keras.layers.Flatten())
        discriminator.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))
        discriminator.summary()
        
        #GAN
        GAN = tf.keras.Sequential([generator, discriminator])
        self.gan = GAN
        
        #compiling
        
        adam_optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate, beta_1=self.beta_1)
        generator.compile(optimizer=adam_optimizer, loss='binary_crossentropy')
        discriminator.compile(optimizer=adam_optimizer, loss='binary_crossentropy')
        GAN.compile(optimizer=adam_optimizer, loss='binary_crossentropy')
        
        return GAN
    
    def training_steps(self):
        
        generator, discriminator = self.gan.layers[0], self.gan.layers[1]
        
        for epoch in range(self.epochs):
            
            #train Discriminator
            #Real samples = 1, fake samples = 0
            discriminator.trainable = True
            
            generated_samples = generator(tf.random.normal(shape=[self.batch_size, self.input_features]))
            real_samples = self.dataset[epoch * self.batch_size : (epoch + 1) * self.batch_size]
            
            discriminator_input = tf.concat([generated_samples, real_samples], axis = 0)
            
            zeros = tf.zeros(( self.batch_size, 1))
            ones = tf.ones(( self.batch_size, 1))
            
            y_train = tf.concat([zeros, ones], axis = 0)
            
            discriminator.train_on_batch(discriminator_input, y_train)
            
            # Training Generator
            discriminator.trainable = False
            
            input_samples = tf.random.normal(shape = [self.batch_size, self.input_features])
            y_train = tf.ones((self.batch_size, 1))
            
            self.gan.train_on_batch(input_samples, y_train)

In [3]:
data = dataset.Dataset().load_data()

In [4]:
data[0]

array([[[0.44313725, 0.38431373, 0.        ],
        [0.50196078, 0.43921569, 0.0627451 ],
        [0.49411765, 0.42745098, 0.0745098 ],
        ...,
        [0.70588235, 0.50588235, 0.00784314],
        [0.70196078, 0.49803922, 0.01960784],
        [0.68235294, 0.48627451, 0.00392157]],

       [[0.45098039, 0.38823529, 0.01176471],
        [0.45882353, 0.4       , 0.03529412],
        [0.41176471, 0.34901961, 0.01568627],
        ...,
        [0.72156863, 0.50980392, 0.01568627],
        [0.70196078, 0.49803922, 0.01960784],
        [0.67058824, 0.4745098 , 0.        ]],

       [[0.45098039, 0.38823529, 0.04313725],
        [0.41176471, 0.35294118, 0.01960784],
        [0.32941176, 0.27843137, 0.        ],
        ...,
        [0.7254902 , 0.51372549, 0.01960784],
        [0.68627451, 0.48235294, 0.01960784],
        [0.65098039, 0.44313725, 0.        ]],

       ...,

       [[0.30588235, 0.42745098, 0.40392157],
        [0.4745098 , 0.58823529, 0.60392157],
        [0.54901961, 0

In [5]:
model = Model()

In [6]:
model.dataset = data

In [7]:
model.GAN()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1600)              161600    
                                                                 
 reshape (Reshape)           (None, 4, 4, 100)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 8, 8, 50)         125050    
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 25)       125025    
 ranspose)                                                       
                                                                 
 conv2d_transpose_2 (Conv2DT  (None, 64, 64, 3)        7503      
 ranspose)                                                       
                                                      

In [ ]:
model.training_steps()

In [ ]:
pic = model.gan.layers[0](tf.random.normal(shape=[1, 100]))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
pic = np.array(pic)

In [ ]:
plt.imshow(pic[0])